In [1]:
from numpy import cos, sin, pi, sqrt, arccos, log
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def simplified_fun(x, y, N):
    #  the boundary is divided into N semgements, 
    #  each segment  end points xp,yp and xm, ym are elements of arrays 
    #  (xp[i],yp[i])__________(barx[i], bary[i])____________(xm[i], ym[i] 
    #  were barx and bary are coordinates of midpoints
    xp = x[1:]
    yp = y[1:]
    xm = x[:-1]
    ym = y[:-1]
    barx = 0.5 * (xp + xm)
    bary = 0.5 * (yp + ym)
    
    ds = sqrt((xp - xm) ** 2 + (yp - ym) ** 2)
    #Compute direction vectors
    directions = np.column_stack([xp - xm, yp - ym])
    # Rotate directions by 90 degrees to obtain normal vectors
    normals = np.column_stack([-directions[:, 1], directions[:, 0]])
    # Normalize normal vectors
    norms = np.linalg.norm(normals, axis=1)
    normals_normalized = normals / norms[:, np.newaxis]
    nn1 = normals_normalized[:, 0]
    nn2 = normals_normalized[:, 1]
    nn6 = barx*nn2 - bary*nn1

    hh1 = np.zeros(N)
    hh2 = np.zeros(N)
    hh6 = np.zeros(N)
    dtheta = np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            a1 = xm[j] - barx[i]
            a2 = ym[j] - bary[i]
            b1 = xp[j] - barx[i]
            b2 = yp[j] - bary[i]

            # we constrain the argument between -1 and 1 to avoid numerical issues due to round-off errors
            ttemp = np.clip((a1 * b1 + a2 * b2) / sqrt( (a1**2 + a2**2) * (b1**2 + b2**2)), -1, 1)
            dtheta[i, j] = -arccos(ttemp)
            if j - i == 0:
                dtheta[i, j] = -pi # for m=n
            x1g = 0.5 * (xp[j] - xm[j]) / sqrt(3) + barx[j]
            y1g = 0.5 * (yp[j] - ym[j]) / sqrt(3) + bary[j]
            x2g = -0.5 * (xp[j] - xm[j]) / sqrt(3) + barx[j]
            y2g = -0.5 * (yp[j] - ym[j]) / sqrt(3) + bary[j]
            hh0 = log((x1g - barx[i])**2 + (y1g - bary[i])**2)
            hh0 = hh0 + log((x2g - barx[i])**2 + (y2g - bary[i])**2)
            hh0 = hh0 * 0.5
            hh1[i] += hh0 * nn1[j] * ds[j] / 2
            hh2[i] += hh0 * nn2[j] * ds[j] / 2
            hh6[i] += hh0 * nn6[j] * ds[j] / 2

    # the linear system is solved using a standard numpy linear algebra solver
    pdcr1 = (np.linalg.solve(dtheta, hh1))
    pdcr2 = (np.linalg.solve(dtheta, hh2))
    pdcr6 = (np.linalg.solve(dtheta, hh6))

    # finally we compute the added mass
    a11 = np.sum(pdcr1 * nn1 * ds)
    a22 = np.sum(pdcr2 * nn2 * ds)
    a66 = np.sum(pdcr6 * nn6 * ds)
   
    return pdcr1, [a11, a22, a66]

In [3]:
def solve(figure_type, a, b, number_of_elems):
    theta = np.arange(0, 2 * pi + 2*pi/number_of_elems, 2*pi/number_of_elems)
    bartheta = (theta[1::]+theta[0:-1])*0.5

    if figure_type == 'eclipse':
        r = a*b/sqrt((b*cos(theta))**2 + (a*sin(theta))**2)
        x = r*cos(theta)
        y = r*sin(theta)
        return simplified_fun(x,y,number_of_elems)

In [4]:
def elipse_m_ij_coeficents(semi_major_len, semi_minor_len, number_of_elems):
    pdcr, a_mass_coef =  solve('eclipse', semi_major_len, semi_minor_len, number_of_elems)
    eccentricity = np.sqrt(1 - (semi_minor_len/semi_major_len)**2)
    m_coef_analytical = np.array([np.pi * semi_minor_len**2, np.pi * semi_major_len**2,
                                  0.125 *np.pi * (semi_major_len**2 - semi_minor_len**2)**2])
    print(f"b_0/a_0 ratio: {semi_minor_len/semi_major_len}")
    print(m_coef_analytical)
    print(a_mass_coef )
    print()


In [5]:
np.set_printoptions(suppress=True)
number_of_elems = 100
elipse_m_ij_coeficents(2, 1, number_of_elems)
elipse_m_ij_coeficents(10, 1, number_of_elems)


b_0/a_0 ratio: 0.5
[ 3.14159265 12.56637061  3.53429174]
[3.1106137501591418, 12.4055562218459, 3.4390460608604263]

b_0/a_0 ratio: 0.1
[   3.14159265  314.15926536 3848.84369973]
[3.0773487950999114, 281.51615733748315, 3063.1751496975257]

